In [1]:
import JSON
using Plots
using HDF5
include("tools.jl")
include("get_grid.jl")
include("get_params.jl")

get_params (generic function with 2 methods)

In [2]:
border = import_border("data/border.json")
dx = 20
Nx, Ny, xrange, yrange, isgrid, isborder, n = get_grid(border, Float64(dx))

x = repeat(reshape(xrange,1,Nx),Ny,1)
y = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .- dx/2
y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2;


In [3]:
recompute_param = true
sigma_l = 100.
if(recompute_param)
    bx, by, p, m, d = get_params(isborder, sigma_l, "data/pantagruel.h5", "grid_params_20_100.h5")
else
    bx, by, p, m, d = get_params("grid_params_20.h5")
end


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [4]:
bx *= dx^2

117×157 Matrix{Float64}:
 0.0  0.0  0.0     0.0     0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0     0.0     0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  2332.6  2872.87     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                               ⋱  ⋮                        ⋮   

In [5]:
by *= dx^2

117×157 Matrix{Float64}:
 0.0  0.0     0.0     0.0      0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0     0.0     0.0      0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  2928.0  3358.41  3817.97     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮           

In [6]:
interval = 200
Niter = 20000
th = zeros(Ny, Nx)

@time begin
    for k in 1:Niter
        dth = 0
        for i in 2:Ny-1
            for j in 2:Nx-1
                if(isgrid[i,j])
                    temp = th[i,j]
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] +
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1])
                    th[i,j] = (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] + 
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1] + dx^2*p[i,j]) / bij
                    dth = max(abs(th[i,j] - temp), dth)
                end
            end
        end   
        if(mod(k,interval) == 0)
            println([k, dth])
        end
    end
end


[200.0, 0.0042301139415032996]
[400.0, 0.0037227836691688942]
[600.0, 0.003152337891562773]
[800.0, 0.0026059987724587064]
[1000.0, 0.0025530377995761633]
[1200.0, 0.002438209486614884]
[1400.0, 0.002266602976690457]
[1600.0, 0.002070486599738963]
[1800.0, 0.0018692501667820771]
[2000.0, 0.0016739329375061729]
[2200.0, 0.0014904049238708694]
[2400.0, 0.0013214163331545237]
[2600.0, 0.0011678734193383278]
[2800.0, 0.0010296213875111704]
[3000.0, 0.0009059213091546781]
[3200.0, 0.0007957387581960518]
[3400.0, 0.0007143949589292209]
[3600.0, 0.0006689400765946818]
[3800.0, 0.0006256177329750923]
[4000.0, 0.0005846016613477723]
[4200.0, 0.0005459641572254093]
[4400.0, 0.0005097069534805154]
[4600.0, 0.00047578358663558085]
[4800.0, 0.0004441155749774772]
[5000.0, 0.00041460408703608564]
[5200.0, 0.00038713831855874403]
[5400.0, 0.0003616014645775323]
[5600.0, 0.0003378749336095943]
[5800.0, 0.00031584127732919853]
[6000.0, 0.0002953861827772464]
[6200.0, 0.00027639978203941595]
[6400.0, 0.

In [9]:
#contour(1:Nx,1:Ny,th, fill=true,levels=-2:0.2:1.6, title="Continuous", colorbar_title = "θ(x,t)")
do_plot(isgrid, th)
savefig("dx20.png")
#do_plot(isgrid, bx)


In [8]:
println(maximum(filter(!isnan,th)))
println(minimum(filter(!isnan,th)))

6.677084812790989
-6.047304844677851
